<a href="https://colab.research.google.com/github/Ate329/ESPGIF_SSD1306/blob/main/TrafficAnalysis/hyperparameters_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install shap
# !pip install scikit-learn==1.3.2
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
import joblib
import time
from sklearn.model_selection import train_test_split
import shap
from matplotlib import pyplot as plt
from sklearn.inspection import permutation_importance
from sklearn.model_selection import cross_val_score, StratifiedKFold

In [ ]:
start_time = time.time()

# Load the dataset
print("Loading the data...")
final_data_path = "final-data.csv"
dataset = pd.read_csv(final_data_path)
print("Successfully loaded the dataset!")

features = dataset[['IPV4_SRC_ADDR', 'L4_SRC_PORT', 'IPV4_DST_ADDR', 'L4_DST_PORT', 'PROTOCOL', 'L7_PROTO', 'IN_BYTES', 'IN_PKTS',
                    'OUT_BYTES', 'OUT_PKTS', 'TCP_FLAGS', 'CLIENT_TCP_FLAGS', 'SERVER_TCP_FLAGS', 'FLOW_DURATION_MILLISECONDS',
                    'MIN_TTL', 'MAX_TTL', 'LONGEST_FLOW_PKT', 'SHORTEST_FLOW_PKT', 'MIN_IP_PKT_LEN', 'MAX_IP_PKT_LEN',
                    'RETRANSMITTED_IN_BYTES', 'RETRANSMITTED_IN_PKTS', 'RETRANSMITTED_OUT_BYTES', 'RETRANSMITTED_OUT_PKTS',
                    'SRC_TO_DST_AVG_THROUGHPUT', 'DST_TO_SRC_AVG_THROUGHPUT']]
target = dataset['Attack']

print("Encoding data...")
features_encoded = pd.get_dummies(features, columns=['IPV4_SRC_ADDR', 'IPV4_DST_ADDR', 'PROTOCOL'])

elapsed_time = time.time() - start_time
print(f"elapsed time to load the data: {elapsed_time}")

Loading the data...
Successfully loaded the dataset!
Encoding data...
elapsed time to load the data: 22.9970645904541


In [ ]:
start_time1 = time.time()

print("loading the model...")
model = joblib.load("weight-RF-model.joblib")

'''
# Use cross-validation to evaluate model performance
print("Calculating cross validation score...")
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
cv_scores = cross_val_score(model, features_encoded, target, cv=cv, scoring='accuracy')
'''
cv_scores = [0.99012875, 0.9900639, 0.99003044, 0.99002416, 0.99011411]

# Check if the model performs well on cross-validation
if all(cv_score > 0.5 for cv_score in cv_scores):  # Adjust the threshold based on your needs
  print(f"Model has reasonable cross-validation performance: {cv_scores}")

  # Now use permutation_importance
  print("calculating permutation importances of the model...")
  result = permutation_importance(model, features_encoded, target, n_repeats=30, random_state=42, n_jobs=1)

  # Extract the importances and feature names
  importances = result.importances_mean
  feature_names = features_encoded.columns  # Assuming you have column names in a DataFrame

  # Sort the indices of importances in descending order
  indices = np.argsort(importances)[::-1]

  # Plot the feature importances
  plt.bar(range(len(indices)), importances[indices], align="center")
  plt.xticks(range(len(indices)), feature_names[indices], rotation='vertical')
  plt.xlabel('Feature')
  plt.ylabel('Permutation Importance')
  plt.show()
else:
  print(f"Model does not perform well on cross-validation. Reevaluate your model. The score is: {cv_scores}")

elapsed_time1 = time.time() - start_time1
print(f"total eslapsed time: {elapsed_time1}")